# Data Acquisition

### Import Libraries


In [1]:
!pip install geemap google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [2]:
import ee
import geemap
import os
import pandas as pd
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt


drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


Mounted at /content/drive


In [3]:
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


### Determining Hotest Summer Days

*Using MODIS hottest cloud-free summer days through 10 year*

In [71]:
def get_modis_top_n_hottest_days(start_year=2014, end_year=2024, n_days=3):
    """
    Finds the top N hottest cloud-free summer days for Hamburg using MODIS LST data.
    Returns a DataFrame where each year maps to a list of dictionaries.
    """
    hamburg = ee.Geometry.Point(9.9937, 53.5511).buffer(5000)
    results = {}

    for year in range(start_year, end_year + 1):
        try:
            modis_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
                .filterBounds(hamburg) \
                .filterDate(f'{year}-05-15', f'{year}-09-15')

            def compute_lst(img):
                mean_lst = img.reduceRegion(reducer=ee.Reducer.mean(), geometry=hamburg, scale=1000).get('LST_Day_1km')
                return ee.Feature(None, {'lst': mean_lst, 'date': img.date().format('YYYY-MM-dd')})

            lst_features = modis_collection.map(compute_lst).filter(ee.Filter.notNull(['lst']))

            if lst_features.size().getInfo() == 0:
                continue

            # Instead of .first(), use .limit(n) to get the top N days
            hottest_list = lst_features.sort('lst', False).limit(n_days).getInfo()['features']

            top_days = []
            for feature in hottest_list:
                props = feature['properties']
                if props['lst'] is not None:
                    celsius = props['lst'] * 0.02 - 273.15
                    top_days.append({
                        'date': props['date'],
                        'lst_celsius': round(celsius, 2)
                    })

            results[year] = {'top_days': top_days}
            print(f"Found top {len(top_days)} days for {year}.")

        except Exception as e:
            print(f"Error processing {year}: {str(e)}")
            continue

    return pd.DataFrame.from_dict(results, orient='index')

print("Extracting MODIS data...")
df_top_days = get_modis_top_n_hottest_days()

Extracting MODIS data...
Found top 7 days for 2014.
Found top 7 days for 2015.
Found top 7 days for 2016.
Found top 7 days for 2017.
Found top 7 days for 2018.
Found top 7 days for 2019.
Found top 7 days for 2020.
Found top 7 days for 2021.
Found top 7 days for 2022.
Found top 7 days for 2023.
Found top 7 days for 2024.


In [72]:
for year, row in df_top_days.iterrows():
    print(f"--- Year: {year} ---")

    # Get the list of candidate days for the current year
    top_days_list = row['top_days']

    # Check if the list is not empty
    if not top_days_list:
        print("  No candidate days found.")
        continue

    # Print each candidate day
    for i, day_info in enumerate(top_days_list):
        date = day_info['date']
        temp = day_info['lst_celsius']
        print(f"  {i+1}. Hottest: Date: {date}, Temp: {temp}°C")
    print("-" * 20)

--- Year: 2014 ---
  1. Hottest: Date: 2014-07-04, Temp: 34.83°C
  2. Hottest: Date: 2014-07-19, Temp: 34.75°C
  3. Hottest: Date: 2014-07-20, Temp: 34.27°C
  4. Hottest: Date: 2014-07-18, Temp: 33.79°C
  5. Hottest: Date: 2014-06-07, Temp: 33.3°C
  6. Hottest: Date: 2014-07-11, Temp: 33.06°C
  7. Hottest: Date: 2014-07-29, Temp: 32.76°C
--------------------
--- Year: 2015 ---
  1. Hottest: Date: 2015-07-05, Temp: 38.17°C
  2. Hottest: Date: 2015-07-02, Temp: 34.78°C
  3. Hottest: Date: 2015-06-12, Temp: 34.33°C
  4. Hottest: Date: 2015-07-04, Temp: 34.18°C
  5. Hottest: Date: 2015-07-07, Temp: 32.98°C
  6. Hottest: Date: 2015-07-01, Temp: 32.9°C
  7. Hottest: Date: 2015-06-05, Temp: 31.52°C
--------------------
--- Year: 2016 ---
  1. Hottest: Date: 2016-06-05, Temp: 35.11°C
  2. Hottest: Date: 2016-06-23, Temp: 34.02°C
  3. Hottest: Date: 2016-07-20, Temp: 32.88°C
  4. Hottest: Date: 2016-06-04, Temp: 32.35°C
  5. Hottest: Date: 2016-06-06, Temp: 32.33°C
  6. Hottest: Date: 2016-07-2

### Extracting Landsat-8 Images Based on Hottest Days

*Landsat 8 images based on hottest days and calculating LTS*

In [87]:
# Hamburg coordinates, define bounding box for missing tiles issue
hamburg = ee.Geometry.Rectangle([
    9.7,  # Min Longitude
    53.4, # Min Latitude
    10.3, # Max Longitude
    53.7  # Max Latitude
])

def get_robust_landsat_data(target_date_str, max_cloud=20, search_radius_days=15):
    """
    Finds a cloud-free Landsat 8 image for a specific target date.
    Returns the image, the found date, and a status ('SUCCESS', 'FALLBACK', 'FAILURE').
    """
    target_date = ee.Date(target_date_str) # Takes a string date instead of a year
    image_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(hamburg)

    # --- Strategy A: Iterative search for a low-cloud image ---
    for day_offset in range(search_radius_days + 1):
        start_date = target_date.advance(-day_offset, 'day')
        end_date = target_date.advance(day_offset, 'day').advance(1, 'day')

        landsat_collection = image_collection \
            .filterDate(start_date, end_date) \
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud)) \
            .sort('CLOUD_COVER')

        if landsat_collection.size().getInfo() > 0:
            found_date_ee = ee.Date(landsat_collection.first().get('system:time_start'))
            found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
            image = landsat_collection.mosaic().clip(hamburg)

            def mask_clouds(img):
                qa = img.select('QA_PIXEL')
                cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
                mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
                return img.updateMask(mask)

            # Return with 'SUCCESS' status
            return mask_clouds(image), found_date_str, 'SUCCESS'

    # --- Strategy B: Fallback if no low-cloud image was found ---
    start_date = target_date.advance(-search_radius_days, 'day')
    end_date = target_date.advance(search_radius_days, 'day').advance(1, 'day')

    fallback_collection = image_collection.filterDate(start_date, end_date).sort('CLOUD_COVER')

    if fallback_collection.size().getInfo() > 0:
        best_image = fallback_collection.first()
        cloud_cover_val = best_image.get('CLOUD_COVER').getInfo()

        # Fallback Threshold: Reject images that are excessively cloudy
        if cloud_cover_val > 60: # You can adjust this threshold
             return None, None, 'FAILURE'

        found_date_ee = ee.Date(best_image.get('system:time_start'))
        found_date_str = found_date_ee.format('YYYY-MM-dd').getInfo()
        image = fallback_collection.mosaic().clip(hamburg)

        def mask_clouds(img):
            qa = img.select('QA_PIXEL')
            cloud_bit_mask = 1 << 3; cloud_shadow_bit_mask = 1 << 4
            mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
            return img.updateMask(mask)

        # Return with 'FALLBACK' status
        return mask_clouds(image), found_date_str, 'FALLBACK'

    # If even the fallback fails, return 'FAILURE'
    return None, None, 'FAILURE'


# Add this helper function to your code
def get_coverage_percentage(image, geometry):
    """Calculates the percentage of valid (unmasked) pixels within a geometry."""
    # Create an image where valid pixels are 1, masked pixels are 0
    valid_pixels = image.select(0).unmask(0).gt(0)

    # Calculate the mean of this binary image within the geometry
    # The mean of a 0/1 image is the percentage of 1s.
    coverage_stats = valid_pixels.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=30, # Landsat scale
        maxPixels=1e9
    )

    # The result is a fraction (0 to 1), so multiply by 100
    return ee.Number(coverage_stats.values().get(0)).multiply(100)

# LST calculation function
def calculate_lst(image):
    # Check if input is a valid ee.Image
    if image is None:
        return None

    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Convert Kelvin to Celsius
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return image.addBands(lst)

In [89]:
years = df_top_days.index.tolist()
lst_images = {}
final_dates = {}

# Process each year one by one
for year in years:
    print(f"\n--- Processing Year: {year} ---")

    # Get the list of hottest candidate days for that year
    top_days_for_year = df_top_days.loc[year, 'top_days']

    successful_candidates = [] # A list to store all successful (non-fallback) results

    # Loop through all candidates for the year to evaluate them
    for i, day_info in enumerate(top_days_for_year):
        target_date = day_info['date']
        print(f"-> Evaluating Candidate #{i+1} | Target Date: {target_date}")

        # Call our robust function to get an image
        landsat_image, found_date, status = get_robust_landsat_data(target_date)

        # We are only interested in high-quality 'SUCCESS' images
        if status == 'SUCCESS':
            # Calculate the percentage of valid pixels for the successful image
            coverage = get_coverage_percentage(landsat_image, hamburg).getInfo()
            print(f"  > Found a SUCCESS candidate. Valid Pixel Coverage: {coverage:.2f}%")

            # Store the result along with its coverage score
            successful_candidates.append({
                'image': landsat_image,
                'found_date': found_date,
                'target_date': target_date,
                'coverage': coverage
            })
        else:
            print(f"  > Candidate resulted in '{status}'. Skipping.")

    # After checking all candidates for the year, decide which one is the best
    if successful_candidates:
        # Sort the successful candidates by their coverage score in descending order
        best_candidate = sorted(successful_candidates, key=lambda x: x['coverage'], reverse=True)[0]

        print(f"BEST OPTION for {year}: Target Date {best_candidate['target_date']} -> Found Image on {best_candidate['found_date']} with {best_candidate['coverage']:.2f}% coverage.")

        # Calculate LST for the best image
        lst_images[year] = calculate_lst(best_candidate['image'])
        final_dates[year] = {
            'target': best_candidate['target_date'],
            'found': best_candidate['found_date'],
            'coverage': best_candidate['coverage']
        }
    else:
        # If no candidates resulted in a 'SUCCESS'
        print(f"FAILURE: Could not find any high-quality image for year {year} from the top candidate days.")


--- Processing Year: 2014 ---
-> Evaluating Candidate #1 | Target Date: 2014-07-04
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
-> Evaluating Candidate #2 | Target Date: 2014-07-19
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
-> Evaluating Candidate #3 | Target Date: 2014-07-20
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.82%
BEST OPTION for 2014: Target Date 2014-07-04 -> Found Image on 2014-07-10 with 99.82% coverage.

--- Processing Year: 2015 ---
-> Evaluating Candidate #1 | Target Date: 2015-07-05
  > Candidate resulted in 'FALLBACK'. Skipping.
-> Evaluating Candidate #2 | Target Date: 2015-07-02
  > Candidate resulted in 'FALLBACK'. Skipping.
-> Evaluating Candidate #3 | Target Date: 2015-06-12
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.73%
BEST OPTION for 2015: Target Date 2015-06-12 -> Found Image on 2015-06-11 with 99.73% coverage.

--- Processing Year: 2016 ---
-> Evaluating Candidate #1 | Target Date: 2016-06-05
  > Foun

  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.43%
-> Evaluating Candidate #3 | Target Date: 2023-06-12
  > Found a SUCCESS candidate. Valid Pixel Coverage: 99.91%
BEST OPTION for 2023: Target Date 2023-06-12 -> Found Image on 2023-06-08 with 99.91% coverage.

--- Processing Year: 2024 ---
-> Evaluating Candidate #1 | Target Date: 2024-06-25
  > Found a SUCCESS candidate. Valid Pixel Coverage: 93.64%
-> Evaluating Candidate #2 | Target Date: 2024-07-20
  > Found a SUCCESS candidate. Valid Pixel Coverage: 14.79%
-> Evaluating Candidate #3 | Target Date: 2024-06-26
  > Found a SUCCESS candidate. Valid Pixel Coverage: 93.64%
BEST OPTION for 2024: Target Date 2024-06-25 -> Found Image on 2024-06-26 with 93.64% coverage.


#### Visualize 10 years LTS data for control

In [90]:
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# Visualization parameters (for single band)
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']
}


# Add each year's LST image to the map
for year, lst_image in lst_images.items():
    try:
        lst_single_band = lst_image.select('LST')
        Map.addLayer(lst_single_band, vis_params, f'LST {year}')
    except Exception as e:
        print(f"{year} error: {str(e)}")

# Add layer control panel
Map.addLayerControl()

map_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/outputs/'
output_path = map_dir + 'LST_map_10years.html'
Map.to_html(output_path)

Problem in 2017 with 67% coverange and 2024 with 93% coverage few missing pixels

*Adding Time Slider to 10 years LTS*


### Extracting Sentinel-2 Images for all summers (Landsat-8 Fallback for missing years)


In [118]:
# Hamburg coordinates, define bounding box for missing tiles issue
hamburg = ee.Geometry.Rectangle([9.7, 53.4, 10.3, 53.7])

# --- LANDSAT 8 IÇIN FONKSIYONLAR ---
def mask_landsat8_clouds(image):
    qa = image.select('QA_PIXEL')
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 4
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
    return image.updateMask(mask)

def create_landsat_summer_composite(year):
    print(f"\n--- Creating LANDSAT-8 composite for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    composite = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .map(mask_landsat8_clouds) \
        .median() \
        .clip(hamburg)
    print(f"Landsat-8 composite successfully created for {year}.")
    return composite

# --- SENTINEL 2 IÇIN FONKSIYONLAR ---
def mask_s2_sr(image):
    scl = image.select('SCL'); good_quality = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(7)); return image.updateMask(good_quality)
def mask_s2_l1c(image):
    qa = image.select('QA60'); c=1<<10; s=1<<11; m=qa.bitwiseAnd(c).eq(0).And(qa.bitwiseAnd(s).eq(0)); return image.updateMask(m)

def create_sentinel2_summer_composite(year):
    if year < 2016: return None

    collection_id = "COPERNICUS/S2_SR_HARMONIZED" if year >= 2017 else "COPERNICUS/S2"
    mask_function = mask_s2_sr if year >= 2017 else mask_s2_l1c

    print(f"\n--- Creating SENTINEL-2 composite for Summer {year} ---")
    start_date = f'{year}-06-01'
    end_date = f'{year}-08-31'

    composite = ee.ImageCollection(collection_id) \
        .filterBounds(hamburg) \
        .filterDate(start_date, end_date) \
        .map(mask_function) \
        .median() \
        .clip(hamburg)
    print(f"Sentinel-2 composite successfully created for {year}.")
    return composite

In [119]:
composite_images = {}
years_to_process = range(2014, 2025)

for year in years_to_process:

    # --- THIS IS THE KEY CHANGE: Use Landsat for 2017 as well ---
    if year < 2016 or year == 2017: # Use Landsat for 2014, 2015, AND 2017
        composite = create_landsat_summer_composite(year)
    else: # For all other years (2016, 2018+), use Sentinel-2
        composite = create_sentinel2_summer_composite(year)

    if composite:
        # Check if the composite image actually has data before adding it
        try:
            # A simple check by trying to get a value. If it fails, the image is empty.
            composite.select(0).reduceRegion(ee.Reducer.first(), hamburg, 1000).getInfo()
            composite_images[year] = composite
        except Exception as e:
            print(f"  > WARNING: Composite for {year} was created but appears to be empty. Skipping. Error: {e}")


--- Creating LANDSAT-8 composite for Summer 2014 ---
Landsat-8 composite successfully created for 2014.

--- Creating LANDSAT-8 composite for Summer 2015 ---
Landsat-8 composite successfully created for 2015.

--- Creating SENTINEL-2 composite for Summer 2016 ---
Sentinel-2 composite successfully created for 2016.

--- Creating LANDSAT-8 composite for Summer 2017 ---
Landsat-8 composite successfully created for 2017.

--- Creating SENTINEL-2 composite for Summer 2018 ---
Sentinel-2 composite successfully created for 2018.

--- Creating SENTINEL-2 composite for Summer 2019 ---
Sentinel-2 composite successfully created for 2019.

--- Creating SENTINEL-2 composite for Summer 2020 ---
Sentinel-2 composite successfully created for 2020.

--- Creating SENTINEL-2 composite for Summer 2021 ---
Sentinel-2 composite successfully created for 2021.

--- Creating SENTINEL-2 composite for Summer 2022 ---
Sentinel-2 composite successfully created for 2022.

--- Creating SENTINEL-2 composite for Summ

#### Visualize Sentinel-2 Data

In [127]:
# Helper function to check if an image has valid data
def is_image_valid(image, geometry):
    """Checks if an image contains any unmasked pixels within the geometry."""
    try:
        stats = image.select(0).reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=100).getInfo()
        return stats and list(stats.values())[0] is not None
    except Exception:
        return False

# Parameters for Sentinel-2
vis_params_s2 = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 100,
    'max': 3500,
    'gamma': 1.4
}

# Corrected parameters for Landsat-8
vis_params_l8_final = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 7000,
    'max': 28000,
    'gamma': 1.4
}

# Create the map
Map = geemap.Map(center=[hamburg.centroid().getInfo()['coordinates'][1], hamburg.centroid().getInfo()['coordinates'][0]], zoom=10)
Map.add_basemap('OpenStreetMap')

for year, image in composite_images.items():
    if not is_image_valid(image, hamburg):
        continue

    # Select the correct parameters and layer name based on the year
    if year < 2016 or year == 2017:
        params = vis_params_l8_final
        layer_name = f'Composite {year} (Landsat-8)'
    else:
        params = vis_params_s2
        layer_name = f'Composite {year} (Sentinel-2)'

    Map.addLayer(image, params, layer_name)

Map.addLayerControl()

# Save the map as an HTML file
map_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/outputs/'
output_path = map_dir + 'CompositeSentinel2Landsat8_map_10years.html'
Map.to_html(output_path)

### Export Images as GeoTIF

In [133]:
# Function to export an image to Google Drive as GeoTIFF
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder.replace('/content/drive/MyDrive/', ''),
        fileNamePrefix=name,
        scale=30,
        region=hamburg.buffer(5000).bounds(),
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

work_dir = 'raw'

export_tasks = []

# Export each year's LST image to Google Drive → landsat_8
for year, lst_image in lst_images.items():
    try:
        task = export_to_drive(lst_image.select('LST'), f'LST_{year}_Hamburg', work_dir)
        export_tasks.append(task)
        print(f"Export task started for {year} (LST).")
    except Exception as e:
        print(f"Error exporting {year} (LST): {str(e)}")

# Wait for all export tasks to complete
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)

print("All export tasks completed!")

Export task started for 2014 (LST).
Export task started for 2015 (LST).
Export task started for 2016 (LST).
Export task started for 2017 (LST).
Export task started for 2018 (LST).
Export task started for 2019 (LST).
Export task started for 2020 (LST).
Export task started for 2021 (LST).
Export task started for 2022 (LST).
Export task started for 2023 (LST).
Export task started for 2024 (LST).


KeyboardInterrupt: 

In [134]:
import ee
import time

# --- 1. Hamburg için tam alanımızı (bounding box) ve hedef klasörü tanımlayalım ---

# Projenin en başından beri kullandığımız tam dikdörtgen alan
hamburg_aoi = ee.Geometry.Rectangle([
    9.7,  # Min Longitude
    53.4, # Min Latitude
    10.3, # Max Longitude
    53.7  # Max Latitude
])

# Google Drive'da dosyaların kaydedileceği tam proje yolu
# Bu yolun kendi Drive yapınla eşleştiğinden emin ol
project_folder_path = 'UHI-Detection-Analysis/data/raw/LST'


# --- 2. Görüntüyü dışa aktaran fonksiyon ---

def export_lst_to_drive(image, name, folder):
    """Exports a single-band LST ee.Image to a specific folder in Google Drive."""
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder,      # Tam ve doğru klasör yolunu kullanır
        fileNamePrefix=name,
        scale=30,           # Landsat'in termal bandının çözünürlüğü
        region=hamburg_aoi, # Buffer olmadan, tam AOI'yi kullanır
        fileFormat='GeoTIFF',
        maxPixels=1e10      # Büyük alanları export ederken limit hatası almamak için
    )
    task.start()
    print(f"  > Export task started for '{name}' -> Folder: {folder}")
    return task


# --- 3. Dışa aktarma işlemlerini başlatalım ---

# lst_images sözlüğünün daha önceki adımlarda dolu olduğunu varsayıyoruz
export_tasks = []
print("Starting LST export tasks...")

for year, lst_image in lst_images.items():
    try:
        # Sadece 'LST' bandını seçtiğimizden emin olalım
        image_to_export = lst_image.select('LST')
        file_name = f'LST_{year}_Hamburg'

        task = export_lst_to_drive(image_to_export, file_name, project_folder_path)
        export_tasks.append(task)
    except Exception as e:
        print(f"  > ERROR starting export for {year} (LST): {str(e)}")


# --- 4. Görevlerin tamamlanmasını bekleyelim ---

print("\nAll tasks started. Monitoring status... (This can take a long time)")
print("You can check the progress in the 'Tasks' tab of the GEE Code Editor.")

while any(task.active() for task in export_tasks):
    active_tasks = sum(1 for task in export_tasks if task.active())
    completed_tasks = sum(1 for task in export_tasks if task.status()['state'] == 'COMPLETED')
    failed_tasks = sum(1 for task in export_tasks if task.status()['state'] == 'FAILED')

    # \r (carriage return) imleci satır başına alır ve üzerine yazar, bu da tek satırda güncellenen bir durum çubuğu oluşturur.
    print(f"Status: {active_tasks} running, {completed_tasks} completed, {failed_tasks} failed.", end='\r')
    time.sleep(30) # Her 30 saniyede bir kontrol et

print("\n\n🎉 All LST export tasks have finished! You can now close this window.")

Starting LST export tasks...
  > Export task started for 'LST_2014_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2015_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2016_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2017_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2018_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2019_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2020_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2021_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2022_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LST_2023_Hamburg' -> Folder: UHI-Detection-Analysis/data/raw/LST
  > Export task started for 'LS

In [130]:
# Function to export an image to Google Drive as GeoTIFF
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder.replace('/content/drive/MyDrive/', ''),  # Relative path for GEE
        fileNamePrefix=name,
        scale=30,
        region=hamburg.buffer(5000).bounds(),
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

work_dir = 'raw'

export_tasks = []

# Export each year's Sentinel-2 image to Google Drive → sentinel_2
for year, s2_image in s2_images.items():
    try:
        task = export_to_drive(s2_image.select(['B4', 'B3', 'B2','B8', 'B11']), f'Sentinel2_{year}_Hamburg', work_dir)
        export_tasks.append(task)
        print(f"Export task started for Sentinel-2 image of {year}.")
    except Exception as e:
        print(f"Error exporting Sentinel-2 image for {year}: {str(e)}")

# Wait for all export tasks to complete
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)

print("All export tasks completed!")

NameError: name 's2_images' is not defined

region = hamburg.buffer(20000).bounds()
Sentinel 2 should be 10m as scale
export location of sentinel 2 is wrong

#### Clean the metadata.widgets Data in the Notebook

In [ ]:
!pip install nbstripout > /dev/null 2>&1

In [ ]:
!nbstripout /content/drive/MyDrive/GitHub_Repos/urban-heat-island/notebooks/01_data_acquisition.ipynb


### Summary and Next Steps

In this notebook, **data acquisition** was prepared to determine the **hottest day** and to download satellite imagery from **Landsat 8 for LST** and **Sentinel-2**.  
These datasets were saved in the `data/raw/` directory.

In the next notebook, **`03_data_processing.ipynb`**, we will combine *LST* with **spectral indices** (e.g., NDVI) to create a **multi-channel tensor**, which will serve as the final input for the **U-Net model**.
